## Calculating similarity score using cosine similarity.

In [81]:
!pip install docx2txt

In [82]:
!pip install strsimpy

In [83]:
# importing all necessary libraries
import pandas as pd
import numpy as np
import docx2txt
import json, collections
from pandas.io.json import json_normalize
import os

In [84]:
!pip install docx2csv

In [85]:
from docx2csv import extract_tables, extract
tables = extract_tables('/content/lists.docx')

In [86]:
import pandas as pd
from docx import Document
document = Document("/content/lists.docx")

In [87]:
def read_docx_table(document,table_num):
  table = document.tables[table_num-1]
  data = [[cell.text for cell in row.cells] for row in table.rows]
  df = pd.DataFrame(data)
  return df

In [88]:
table_num=1
df = read_docx_table(document,table_num)
df.head()

,0,1
0,P1,Analyze a complex computing problem and to app...
1,P2,"Design, implement, and evaluate a computing-ba..."
2,P3,Communicate effectively in a variety of profes...
3,P4,Recognize professional responsibilities and ma...
4,P5,Function effectively as a member or leader of ...


In [89]:
q1 = df.copy()

In [90]:
table_num=2
df1 = read_docx_table(document,table_num)
df1.head()

,0,1
0,C1,Explain how AI impacts society
1,C2,Apply AI methods to perform practical tasks
2,C3,Synthesize a simple AI system
3,C4,Evaluate different AI approaches to solve a pr...
4,C5,Explain the basic principles of the data minin...


In [91]:
P1 = ["Analyze a complex computing problem and to apply principles of computing and other relevant disciplines to identify solutions."]
p1 = df1.copy()
p1.head()

,0,1
0,C1,Explain how AI impacts society
1,C2,Apply AI methods to perform practical tasks
2,C3,Synthesize a simple AI system
3,C4,Evaluate different AI approaches to solve a pr...
4,C5,Explain the basic principles of the data minin...


In [92]:
df1 = pd.DataFrame({
    "name": ["mahesh", "suresh"]
})

df2 = pd.DataFrame({
    "name": ["mahesh", "surendra", "shrivatsa", "suresh", "maheshwari"]
})

df = pd.MultiIndex.from_product(
    [df1["name"], df2["name"]], names=["col1", "col2"]
).to_frame(index=False)


In [93]:
d = pd.MultiIndex.from_product(
    [q1[1], p1[1]], names=["col1", "col2"]
).to_frame(index=False)

In [94]:
d.head()

,col1,col2
0,Analyze a complex computing problem and to app...,Explain how AI impacts society
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...


In [95]:
d.shape

(786, 2)

In [96]:
d.head(133)

,col1,col2
0,Analyze a complex computing problem and to app...,Explain how AI impacts society
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...
...,...,...
128,Analyze a complex computing problem and to app...,Define the computing requirements appropriate ...
129,Analyze a complex computing problem and to app...,Apply knowledge of computer science theory and...
130,Analyze a complex computing problem and to app...,Develop a computer science application.
131,"Design, implement, and evaluate a computing-ba...",Explain how AI impacts society


In [97]:
from strsimpy.cosine import Cosine
cosine = Cosine(2)
d["p0"] = d["col1"].apply(lambda s: cosine.get_profile(s)) 
d["p1"] = d["col2"].apply(lambda s: cosine.get_profile(s)) 
d["cosine_sim"] = [cosine.similarity_profiles(p0,p1) for p0,p1 in zip(d["p0"],d["p1"])]



In [98]:
d.drop(["p0", "p1"], axis=1, inplace=True)
d.head()

,col1,col2,cosine_sim
0,Analyze a complex computing problem and to app...,Explain how AI impacts society,0.208500
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks,0.303507
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system,0.299538
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...,0.442296
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...,0.479283


In [99]:
d.head()

,col1,col2,cosine_sim
0,Analyze a complex computing problem and to app...,Explain how AI impacts society,0.208500
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks,0.303507
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system,0.299538
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...,0.442296
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...,0.479283


In [100]:
d.tail()

,col1,col2,cosine_sim
781,Apply computer science theory and software dev...,Discuss the socio-economic implications of onl...,0.439027
782,Apply computer science theory and software dev...,Function in groups to assess current ethical i...,0.409841
783,Apply computer science theory and software dev...,Define the computing requirements appropriate ...,0.704925
784,Apply computer science theory and software dev...,Apply knowledge of computer science theory and...,0.636566
785,Apply computer science theory and software dev...,Develop a computer science application.,0.557007


In [101]:
def split_dataframe(df, chunk_size = 131): 
    chunks = list()
    num_chunks = len(df) // chunk_size + (1 if len(df) % chunk_size else 0)
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [102]:
d_split2 = split_dataframe(d, chunk_size=131)

In [103]:
d_split2

[                                                  col1  ... cosine_sim
 0    Analyze a complex computing problem and to app...  ...   0.208500
 1    Analyze a complex computing problem and to app...  ...   0.303507
 2    Analyze a complex computing problem and to app...  ...   0.299538
 3    Analyze a complex computing problem and to app...  ...   0.442296
 4    Analyze a complex computing problem and to app...  ...   0.479283
 ..                                                 ...  ...        ...
 126  Analyze a complex computing problem and to app...  ...   0.483211
 127  Analyze a complex computing problem and to app...  ...   0.447032
 128  Analyze a complex computing problem and to app...  ...   0.637084
 129  Analyze a complex computing problem and to app...  ...   0.617056
 130  Analyze a complex computing problem and to app...  ...   0.493144
 
 [131 rows x 3 columns],
                                                   col1  ... cosine_sim
 131  Design, implement, and evaluate

In [104]:
d_split2[0].head()

,col1,col2,cosine_sim
0,Analyze a complex computing problem and to app...,Explain how AI impacts society,0.208500
1,Analyze a complex computing problem and to app...,Apply AI methods to perform practical tasks,0.303507
2,Analyze a complex computing problem and to app...,Synthesize a simple AI system,0.299538
3,Analyze a complex computing problem and to app...,Evaluate different AI approaches to solve a pr...,0.442296
4,Analyze a complex computing problem and to app...,Explain the basic principles of the data minin...,0.479283


In [105]:
d_split2[1].head()

,col1,col2,cosine_sim
131,"Design, implement, and evaluate a computing-ba...",Explain how AI impacts society,0.265864
132,"Design, implement, and evaluate a computing-ba...",Apply AI methods to perform practical tasks,0.255074
133,"Design, implement, and evaluate a computing-ba...",Synthesize a simple AI system,0.368307
134,"Design, implement, and evaluate a computing-ba...",Evaluate different AI approaches to solve a pr...,0.454319
135,"Design, implement, and evaluate a computing-ba...",Explain the basic principles of the data minin...,0.471455


In [106]:
d_split2[2].head()

,col1,col2,cosine_sim
262,Communicate effectively in a variety of profes...,Explain how AI impacts society,0.176304
263,Communicate effectively in a variety of profes...,Apply AI methods to perform practical tasks,0.227476
264,Communicate effectively in a variety of profes...,Synthesize a simple AI system,0.203531
265,Communicate effectively in a variety of profes...,Evaluate different AI approaches to solve a pr...,0.327247
266,Communicate effectively in a variety of profes...,Explain the basic principles of the data minin...,0.338692


In [107]:
d_split2[3].head()

,col1,col2,cosine_sim
393,Recognize professional responsibilities and ma...,Explain how AI impacts society,0.220880
394,Recognize professional responsibilities and ma...,Apply AI methods to perform practical tasks,0.350757
395,Recognize professional responsibilities and ma...,Synthesize a simple AI system,0.271991
396,Recognize professional responsibilities and ma...,Evaluate different AI approaches to solve a pr...,0.321048
397,Recognize professional responsibilities and ma...,Explain the basic principles of the data minin...,0.529500


In [108]:
d_split2[4].tail()

,col1,col2,cosine_sim
650,Function effectively as a member or leader of ...,Discuss the socio-economic implications of onl...,0.443129
651,Function effectively as a member or leader of ...,Function in groups to assess current ethical i...,0.496404
652,Function effectively as a member or leader of ...,Define the computing requirements appropriate ...,0.535413
653,Function effectively as a member or leader of ...,Apply knowledge of computer science theory and...,0.568737
654,Function effectively as a member or leader of ...,Develop a computer science application.,0.447214


In [109]:
d_split2[5].head()

,col1,col2,cosine_sim
655,Apply computer science theory and software dev...,Explain how AI impacts society,0.178410
656,Apply computer science theory and software dev...,Apply AI methods to perform practical tasks,0.318730
657,Apply computer science theory and software dev...,Synthesize a simple AI system,0.288348
658,Apply computer science theory and software dev...,Evaluate different AI approaches to solve a pr...,0.410004
659,Apply computer science theory and software dev...,Explain the basic principles of the data minin...,0.333951


In [110]:
d_split2[0].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[1].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[2].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[3].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[4].drop(["col1", "col2"], axis=1, inplace=True)
d_split2[5].drop(["col1", "col2"], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [111]:
d_split2[0].rename(columns = {'cosine_sim':'P1'}, inplace = True)
d_split2[0].head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,P1
0,0.208500
1,0.303507
2,0.299538
3,0.442296
4,0.479283


In [112]:
d_split2[1].rename(columns = {'cosine_sim':'P2'}, inplace = True)
d_split2[2].rename(columns = {'cosine_sim':'P3'}, inplace = True)
d_split2[3].rename(columns = {'cosine_sim':'P4'}, inplace = True)
d_split2[4].rename(columns = {'cosine_sim':'P5'}, inplace = True)
d_split2[5].rename(columns = {'cosine_sim':'P6'}, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [113]:
d_split2[5].reset_index(drop=True, inplace=True)
d_split2[5].head()

,P6
0,0.178410
1,0.318730
2,0.288348
3,0.410004
4,0.333951


In [114]:
d_split2[1].reset_index(drop=True, inplace=True)
d_split2[2].reset_index(drop=True, inplace=True)
d_split2[3].reset_index(drop=True, inplace=True)
d_split2[4].reset_index(drop=True, inplace=True)

In [115]:
dd = pd.concat([d_split2[0], d_split2[1], d_split2[2], d_split2[3], d_split2[4], d_split2[5]], axis=1)
dd.head()

,P1,P2,P3,P4,P5,P6
0,0.208500,0.265864,0.176304,0.220880,0.196960,0.178410
1,0.303507,0.255074,0.227476,0.350757,0.385752,0.318730
2,0.299538,0.368307,0.203531,0.271991,0.230409,0.288348
3,0.442296,0.454319,0.327247,0.321048,0.492093,0.410004
4,0.479283,0.471455,0.338692,0.529500,0.442406,0.333951


In [116]:
dd.tail()

,P1,P2,P3,P4,P5,P6
126,0.483211,0.514038,0.423514,0.500397,0.443129,0.439027
127,0.447032,0.562750,0.468645,0.623378,0.496404,0.409841
128,0.637084,0.740578,0.376165,0.488727,0.535413,0.704925
129,0.617056,0.501875,0.394985,0.519288,0.568737,0.636566
130,0.493144,0.462182,0.356529,0.323902,0.447214,0.557007


In [117]:
df2 = pd.concat([p1[0], dd], axis=1)
df2.head()

,0,P1,P2,P3,P4,P5,P6
0,C1,0.208500,0.265864,0.176304,0.220880,0.196960,0.178410
1,C2,0.303507,0.255074,0.227476,0.350757,0.385752,0.318730
2,C3,0.299538,0.368307,0.203531,0.271991,0.230409,0.288348
3,C4,0.442296,0.454319,0.327247,0.321048,0.492093,0.410004
4,C5,0.479283,0.471455,0.338692,0.529500,0.442406,0.333951


In [118]:
df2.set_index(0, inplace=True)
df2.head()

,P1,P2,P3,P4,P5,P6
0,,,,,,
C1,0.208500,0.265864,0.176304,0.220880,0.196960,0.178410
C2,0.303507,0.255074,0.227476,0.350757,0.385752,0.318730
C3,0.299538,0.368307,0.203531,0.271991,0.230409,0.288348
C4,0.442296,0.454319,0.327247,0.321048,0.492093,0.410004
C5,0.479283,0.471455,0.338692,0.529500,0.442406,0.333951


In [119]:
df2.tail()

,P1,P2,P3,P4,P5,P6
0,,,,,,
C127,0.483211,0.514038,0.423514,0.500397,0.443129,0.439027
C128,0.447032,0.562750,0.468645,0.623378,0.496404,0.409841
C129,0.637084,0.740578,0.376165,0.488727,0.535413,0.704925
C130,0.617056,0.501875,0.394985,0.519288,0.568737,0.636566
C131,0.493144,0.462182,0.356529,0.323902,0.447214,0.557007
